### Lastly, GRU!

This particulat example has been taken from Lasagne git repo and modified to suit the purpose!<br>
https://github.com/Lasagne/Recipes/blob/master/examples/lstm_text_generation.py

In [1]:
import numpy as np
import theano
import theano.tensor as T
import lasagne

Using gpu device 0: Tesla K40c (CNMeM is disabled, cuDNN 5103)
/home/aashraya/.env/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


### Data
Data is random text file form web. <br>
Available at https://s3.amazonaws.com/text-datasets/nietzsche.txt

In [2]:
with open("./data/nietzsche.txt") as f:
    in_text = f.read()

In [3]:
# Seed text
seed_text = "Studying at Indian Institute of Science is like" 

In [4]:
#distinct characters
chars = list(set(in_text))

In [5]:
data_size = len(in_text)
vocab_size = len(chars)
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

In [6]:
seq_length = 20
epochs = 50
batch_size = 128

In [7]:
def gen_data(p, batch_size = batch_size, data=in_text, flag = True):
    
    #create one-hot vectors!!
    
    x = np.zeros((batch_size,seq_length,vocab_size))
    y = np.zeros(batch_size)

    for n in range(batch_size):
        ptr = n
        for i in range(seq_length):
            x[n,i,char_to_ix[data[p+ptr+i]]] = 1.
        if flag:
            y[n] = char_to_ix[data[p+ptr+seq_length]]
    return x, np.array(y,dtype='int32')

In [9]:
#batch_size X seq_len x vocab_len

input_layer = lasagne.layers.InputLayer(shape=(None, None, vocab_size))

#BPTT is done automaatically for seq_len time steps
gru_1 = lasagne.layers.GRULayer(input_layer, 
                                num_units=512, 
                                grad_clipping=100)

gru_2 = lasagne.layers.GRULayer(gru_1, 
                                 num_units = 512, 
                                 grad_clipping=100,
                                 only_return_final=True)


output_layer = lasagne.layers.DenseLayer(gru_2, 
                                  num_units=vocab_size, 
                                  W = lasagne.init.Normal(), 
                                  nonlinearity=lasagne.nonlinearities.softmax)

In [10]:
target_values = T.ivector('target_output')

In [11]:
network_output = lasagne.layers.get_output(output_layer)

cost = T.nnet.categorical_crossentropy(network_output,target_values)
cost = cost.mean()
all_params = lasagne.layers.get_all_params(output_layer,trainable=True)

updates = lasagne.updates.adam(cost, all_params)

In [12]:
train = theano.function([input_layer.input_var, target_values], 
                        cost, 
                        updates=updates, 
                        allow_input_downcast=True)

probs = theano.function([input_layer.input_var],
                        network_output,
                        allow_input_downcast=True)

In [15]:
def try_it_out(N=200):
    
    sample_ix = []
    
    x,_ = gen_data(len(seed_text)- seq_length, 1, seed_text,0)

    for i in range(N):
        ix = np.argmax(probs(x).ravel())
        sample_ix.append(ix)
        x[:,0:seq_length-1,:] = x[:,1:,:]
        x[:,seq_length-1,:] = 0
        x[0,seq_length-1,sample_ix[-1]] = 1. 

    random_snippet = seed_text + ''.join(ix_to_char[ix] for ix in sample_ix)    
    print "..............\n", random_snippet ,"\n,,,,,,,,,,,,"

In [16]:
p = 0
for epoch in xrange(data_size * epochs / batch_size):
    try_it_out() 
    avg_cost = 0;
    for _ in range(1000):
        x,y = gen_data(p)
        p += seq_length + batch_size - 1 
        
        if(p+batch_size+seq_length >= data_size):
            p = 0
            
        avg_cost += train(x, y)
    print "Epoch", epoch*1.0*1000/data_size*batch_size, "average loss",avg_cost / 1000

..............
Studying at Indian Institute of Science is like����-g6--------���ttttFzz4UU1Ur"""LLLLLLkkkQD LQ6DDDDD???EQ,ttttt��tUUtttttUL222222222222222ZZZwwwwppppppSSSwwNNwwe"""BBBBXFFqjk,,SS,����1111JJr)aaaaasaBBBrr::::(E,S??RR111RRwwwhhoooCCv�(OO__iBAAttttt 
,,,,,,,,,,,,
Epoch 0.0 average loss 2.38199569547
..............
Studying at Indian Institute of Science is like the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the sust and the  
,,,,,,,,,,,,
Epoch 0.213013458124 average loss 1.79037140739
..............
Studying at Indian Institute of Science is like the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the Jeart of the  
,,,,,,,,,,,,
Epoch 0.426026916247 average loss 1.64060929787
..............
Studying at Indian In

KeyboardInterrupt: 

#### Studying at Indian Institute of Science is like Platonic and sensations of the powerful man is a man in the same and in the same and the soul and simple the question of the soul and as a people who are strongest and invertion of the soul